## Numerical integration: Monte Carlo vs the Determinists

- Implement a Monte Carlo integration to a function in one or more dimensions.

- Quantify the accuracy of your MC integral as a function of the number of sample points and compare with the accuracy of your deterministic methods

## My function of choice: $ f(x) = xsin(x)$ 
 - Derivative: $ f'(x) = sin(x) + xcos(x) $
 
 - Integral: $\int \mathrm{xsin(x)} \mathrm{d}x$ = $ sin(x) - xcos(x) $
 


 # Methods:
 
 - Midpoint rule
 - Gaussian Quadrature
 - Monte Carlo
 - Trapizoidal 

In [36]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.special import legendre
from numpy.polynomial.legendre import legroots as leg
from scipy.special import roots_legendre as rootleg
import Random as Random

# Our function to be integrated, takes a value
def Function(value):
    return(value*np.sin(value))


# rule that numerically integrates according: takes a val, a lower bound and upper bound to be intgrated 
def Midpoint(step, low, high):
    
    value1 = []
    value2 = 0
    width  = (high-low)/step  
     
    for n in range (1,step+1):                      #to offset the initial value 0 which python counts from.
        sub_int_mid = low + (n-1/2)*width           #Midpoint of each subinterval
        value1.append(Function(sub_int_mid))
    value2          = width*np.sum(value1)          #approximated area
    return(value2)

#Implementation of trapazoidal rule
def Trap(step, low, high):
    width = (high-low)/step
    value = [Function(low)]
    
    for i in range(1,step):
        value.append(2*Function(low + i*width))
    value.append(Function(high))
    return(0.5*width*np.sum(value))

#Implementation of Gaussian quadrature on arbitrary intervals
def Gauss(step, low, high):
    width  = 0.5*(low + high)
    roots  = rootleg(step)[0]
    weights = rootleg(step)[1]
    value  = []
    
    for i in range(len(roots)):
        t = roots[i]
        legen = 0.5*(high-low)*weights[i]*Function(0.5*(low + high) + 0.5*(high-low)*t)
        value.append(legen)
    return(np.sum(value)) 

#Calculates the relative error of the different rules (fun)
def Relative_Error(fun, step, low, high):
    integral_high   = np.sin(high)-high*np.cos(high)
    integral_low    = np.sin(low)-low*np.cos(low)
    integral        = integral_high - integral_low
    return(abs(integral-fun(step,low,high))/integral)*100


def Plot(f, step, low, high):
    
    X_Values  = []
    Y_Values  = []
    Rel_Error = []
    
    
    for i in range(1,step+1):
        a   = f(i,low,high)
        err = Relative_Error(f, i, low, high)
        X_Values.append(i)
        Y_Values.append(a)
        Rel_Error.append(err)
    

    fig   = plt.figure()
    axes1 = fig.add_axes([0.1, 0.1, 0.8, 0.8]) # main axes
    axes2 = fig.add_axes([0.50, 0.25, 0.40, 0.35],facecolor='y') # inset axes
    axes1.plot(X_Values, Y_Values,'go--')
    axes2.plot(X_Values, Rel_Error,'b')
    axes1.set_title("Value Vs Steps")
    axes2.set_title("Error Vs Steps")
    axes1.set_xlabel('# Of Sub-Interval')
    axes2.set_xlabel('# Of Sub-Interval')
    axes1.set_ylabel('Value')
    axes2.set_ylabel('Value')
    plt.show()

# New Monte Carlo Implmentation

In [276]:
def Monte_Carlo(sample,low,high):
    value = []
    
    for _ in range(sample):
        x = np.random.rand()
        value.append(Function(x))
    return(high-low)*sum(value)/sample

def MC_variance(sample,high):
    int_max = high # this is the max of our integration range
    
    # get the average of squares
    running_total = 0
    for i in range(sample):
        x = np.random.rand()
        running_total += Function(x)
    sum_of_sqs = running_total*int_max/sample
    
    # get square of average
    running_total = 0
    for i in range(sample):
        x = np.random.rand()
        running_total += Function(x)
    sq_ave = (int_max*running_total/sample)**2
    
    return sum_of_sqs - sq_ave


In [296]:
print("The monte carlo evaluation is:" + str(Monte_Carlo(5000,2,7)))
print("The monte carlo variance is:" + str(MC_variance(5000,7)))

print("The value of of the integral using the trap method is: " + str(Trap(50,2,7)))
print("The relative error of the trap method is: "+ str(Relative_Error(Trap,50,2,7))) 

The monte carlo evaluation is:1.525916951586139
The monte carlo variance is:-2.4249839002606577
The value of of the integral using the trap method is: -6.357038455828771
The relative error of the trap method is: -0.07673509820067714


# Discussion:

Unless it's boostrapped, the monte carlo method is near useless for certain classes of function. 
The deterministic models win hands down. I've used different distributions -normal being the better of them - which sorta made it better, but the value still fluctuated greatly. Also, if one is stuck in a solution region, it doesn't matter the number of sample.  Moreover, for the more special of distributions - such as a normal - I needed ot know a prior where to set sigma and mu. Whereas the deterministic methods simply needs more steps. 